Using kenpom websites to make predictions creates the problem of leakage because they update their stats after the tournament end. Therefore, in this notebook, I decided to scrap kenpom data before tournament for each year. I did it between 2011 to 2022. If you find archives for earlier years please put it in comment section.

Also, earlier years may not work because of the pyth number which is incompatible with anything Ken publishes today.

# Imports

In [1]:
# !pip install bs4

In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

# Scraper

In [3]:
base_urls=[
          'https://web.archive.org/web/20110311233233/http://www.kenpom.com/',
          'https://web.archive.org/web/20120311165019/http://kenpom.com/',
          'https://web.archive.org/web/20130318221134/http://kenpom.com/',
          'https://web.archive.org/web/20140318100454/http://kenpom.com/',
          'https://web.archive.org/web/20150316212936/http://kenpom.com/',
          'https://web.archive.org/web/20160314134726/http://kenpom.com/',
          'https://web.archive.org/web/20170312131016/http://kenpom.com/',
          'https://web.archive.org/web/20180311122559/https://kenpom.com/',
          'https://web.archive.org/web/20190317211809/https://kenpom.com/',
          # Tournament wasn't played on 2020
          'https://web.archive.org/web/20210314233855/http://kenpom.com/',
          'https://web.archive.org/web/20220322041529/https://kenpom.com/',
          'https://web.archive.org/web/20230314032131/https://kenpom.com/',
          'https://web.archive.org/web/20240319075254/https://kenpom.com/',
          'https://www.kenpom.com/'
         ]

def scrap_archive(url,year):
    """
    Imports raw data from a kenpom archive into a dataframe
    """
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    table_full = soup.find_all('table', {'id': 'ratings-table'})

    thead = table_full[0].find_all('thead')
    table = table_full[0]
    
    for weird in thead:
        table = str(table).replace(str(weird), '')

    df = pd.read_html(table)[0]
    df['year'] = year
    
    return df
    

# Scraping

In [4]:
def scraping(df,year):
    
    for url in base_urls:
    
        print(f'Scrapping: {url}')
        archive=scrap_archive(url,year)
        
        df = pd.concat( (df, archive), axis=0) 
        year+=1
    df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
    
    # df=df[[ 'Year', 'Team', 'AdjustO', 'AdjustD', 'AdjustT', 'Luck', 'SOS OppO', 'SOS OppD']]
    # df.columns=[ 'Season', 'TeamName', 'adj_o', 'adj_d', 'adj_tempo', 'luck', 'sos_adj_o', 'sos_adj_d']

    # df.TeamName=df.TeamName.apply(lambda x: re.sub('\d', '', x).strip()).replace('.','')
             
        
    return df

In [5]:
df=None
year=2011
df=scraping(df,year)
df.head()

Scrapping: https://web.archive.org/web/20110311233233/http://www.kenpom.com/


/tmp/ipykernel_16479/153698493.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]


Scrapping: https://web.archive.org/web/20120311165019/http://kenpom.com/
Scrapping: https://web.archive.org/web/20130318221134/http://kenpom.com/


/tmp/ipykernel_16479/153698493.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]


ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20130318221134/http://kenpom.com/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ffac8dc0970>: Failed to establish a new connection: [Errno 111] Connection refused'))

Unfortunately, I didn't find any archives for years between 2003 and 2010 ... If someone has a link to these archives please put it in comment section, it will help all of us to avoid data leakage. 